In [61]:
from PIL import Image
import glob
import os

In [62]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# train 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path_rock = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
image_dir_path_scissor = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
image_dir_path_paper = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
resize_images(image_dir_path_rock)
resize_images(image_dir_path_scissor)
resize_images(image_dir_path_paper)

# test 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path_test_rock = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/rock"
image_dir_path_test_scissor = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/scissor"
image_dir_path_test_paper = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/paper"
resize_images(image_dir_path_test_rock)
resize_images(image_dir_path_test_scissor)
resize_images(image_dir_path_test_paper)

print("이미지 resize 완료!")

575  images to be resized.
575  images resized.
548  images to be resized.
548  images resized.
600  images to be resized.
600  images resized.
300  images to be resized.
300  images resized.
300  images to be resized.
300  images resized.
300  images to be resized.
300  images resized.
이미지 resize 완료!


In [63]:
import numpy as np

def load_data(img_path, number_of_data=1723):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train), (x_test, y_test)=load_data(image_dir_path), load_data(image_dir_path)
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 1723 입니다.
학습데이터(x_train)의 이미지 개수는 1723 입니다.
x_train shape: (1723, 28, 28, 3)
y_train shape: (1723,)


In [64]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

first_Conv2D = 16
second_Conv2D = 64
Dense = 64

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(first_Conv2D, (3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Conv2D(second_Conv2D, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(Dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))
model.summary()

Model에 추가된 Layer 개수:  7
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 11, 11, 64)        9280      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)                102464    
_________________________________________________________________
dense_21 (Dense)             

In [65]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)

Epoch 1/10
54/54 [==============================] - 1s 4ms/step - loss: 11.6037 - accuracy: 0.6233
Epoch 2/10
54/54 [==============================] - 0s 3ms/step - loss: 0.0537 - accuracy: 0.9878
Epoch 3/10
54/54 [==============================] - 0s 3ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 4/10
54/54 [==============================] - 0s 3ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 5/10
54/54 [==============================] - 0s 3ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 6/10
54/54 [==============================] - 0s 3ms/step - loss: 6.8519e-04 - accuracy: 1.0000
Epoch 7/10
54/54 [==============================] - 0s 3ms/step - loss: 4.6555e-04 - accuracy: 1.0000
Epoch 8/10
54/54 [==============================] - 0s 3ms/step - loss: 3.2894e-04 - accuracy: 1.0000
Epoch 9/10
54/54 [==============================] - 0s 3ms/step - loss: 2.5937e-04 - accuracy: 1.0000
Epoch 10/10
54/54 [==============================] - 0s 3ms/step - loss: 2.3830e-04 - accuracy: 1.000

In [66]:
test_loss, test_accuracy = model.evaluate(x_train_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

54/54 - 0s - loss: 1.0595 - accuracy: 1.0000
test_loss: 1.0595017671585083 
test_accuracy: 1.0
